CUDA_VISIBLE_DEVICES=0 python bb.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  TEST.IMS_PER_BATCH 1 DTYPE "float16" GLOVE_DIR /home/suji/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/suji/checkpoints/causal-motifs-sgcls-exmp OUTPUT_DIR /home/suji/checkpoints/causal-motifs-sgcls-exmp

In [1]:

cmdargs = """MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  TEST.IMS_PER_BATCH 1 DTYPE "float16" GLOVE_DIR /home/suji/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/suji/checkpoints/causal-motifs-sgcls-exmp OUTPUT_DIR /home/suji/checkpoints/causal-motifs-sgcls-exmp"""
cmdargs = cmdargs.split()

In [2]:
len(cmdargs)

22

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.config import cfg
import torch

cfg.merge_from_file("configs/e2e_relation_X_101_32_8_FPN_1x.yaml")
cfg.merge_from_list(cmdargs)

# cfg.merge_from_list(None)
cfg.freeze()

# logger.info("Using {} GPUs".format(num_gpus))
# logger.info(cfg)

# logger.info("Collecting env info (might take some time)")
# logger.info("\n" + collect_env_info())

model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)
# Initialize mixed-precision if necessary
use_mixed_precision = cfg.DTYPE == 'float16'
# amp_handle = amp.init(enabled=use_mixed_precision, verbose=cfg.AMP_VERBOSE)

output_dir = "/home/suji/checkpoints/causal-motifs-sgcls-exmp"
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
_ = checkpointer.load(cfg.MODEL.WEIGHT)


RPNModule
loading word vectors from /home/suji/glove/glove.6B.200d.pt
__background__ -> __background__ 
fail on __background__
loading word vectors from /home/suji/glove/glove.6B.200d.pt
__background__ -> __background__ 
fail on __background__


In [5]:
from maskrcnn_benchmark.data import *
data_loaders_val =  make_vcr_data_loader(cfg, is_distributed=False)



paths_catalog, vcr


In [6]:
cfg.DATASETS.VCR
data_loaders_val

In [7]:
from tqdm import tqdm
import pandas as pd

dataset_names = cfg.DATASETS.VCR
device = torch.device("cuda")
idslist = []
proplist = []
feature_list = []
outlist = []
origfeatlist = []
for dataset_name, data_loader in zip(dataset_names, data_loaders_val):
    model.eval()
    results_dict = {}
    cpu_device = torch.device("cpu")
    torch.cuda.empty_cache()
    for _, batch in enumerate(tqdm(data_loader)):
        with torch.no_grad():
            images, targets, image_ids = batch
            output, proposal, feature, orig_features = model(images.to(device), None, objdet = True)
            print("img_id", image_ids)
            print("proposal:", proposal)
            idslist.append(image_ids)
            proplist.append(proposal)
            feature_list.append(feature)
            outlist.append(output)
            origfeatlist.append(orig_features)
#             df = df.append({'img_id': image_ids,'proposals': proposal})
#             print(proposal)
            #     output = [o.to(cpu_device) for o in output]
            results_dict.update(
                {img_id: result for img_id, result in zip(image_ids, output)}
            )
    torch.cuda.empty_cache()


  0%|          | 0/100 [00:00<?, ?it/s]

/home/suji/spring20/vilbert_beta/data/VCR/vcr1images/lsmdc_1054_Harry_Potter_and_the_prisoner_of_azkaban/1054_Harry_Potter_and_the_prisoner_of_azkaban_01.20.34.896-01.20.49.435@3.jpg  box tensor([[2083.4241,  289.5388, 2615.3529, 1456.6500],
        [2532.8556,  154.8922, 3413.6167, 1443.2064],
        [1098.0303,  275.6889, 1923.9681, 1465.1793],
        [1412.0704,  904.1848, 2094.2500, 1473.3202],
        [1768.0912,  945.2669, 2120.6536, 1476.7866]], dtype=torch.float64)/home/suji/spring20/vilbert_beta/data/VCR/vcr1images/lsmdc_1054_Harry_Potter_and_the_prisoner_of_azkaban/1054_Harry_Potter_and_the_prisoner_of_azkaban_00.01.46.736-00.01.50.168@0.jpg  box tensor([[1792.0159,   98.1179, 2909.1895, 1479.4361],
        [  38.2983,   61.1126, 1718.4752, 1475.9934],
        [1691.2625,  209.3555, 1941.1649, 1314.9911],
        [2630.8369,  529.1337, 2892.0204, 1045.1114],
        [1472.8781,  989.5760, 1577.5107, 1232.6794],
        [ 687.6988,    0.0000,  913.4336,  148.6710]], dtype=to

  1%|          | 1/100 [00:00<01:18,  1.27it/s]

/home/suji/spring20/vilbert_beta/data/VCR/vcr1images/lsmdc_0050_Indiana_Jones_and_the_last_crusade/0050_Indiana_Jones_and_the_last_crusade_01.01.42.002-01.01.44.662@0.jpg  box tensor([[2697.2807,  567.1021, 3176.4393, 1529.4889],
        [1651.7601,  571.4487, 1960.8261, 1289.6941],
        [1924.2270,  605.0742, 2223.9825, 1285.0982],
        [1936.5550,  532.8686, 2107.4231, 1118.3462],
        [2985.1893,  899.7049, 3184.3499, 1123.3969]], dtype=torch.float64) torch.Size([5, 4])
0050_Indiana_Jones_and_the_last_crusade_01.01.42.002-01.01.44.662@0.jpg  target after BoxList(num_boxes=5, image_width=999, image_height=429, mode=xyxy)
reln 1
orig_features torch.Size([21, 4096])

img_id (0,)
proposal: [BoxList(num_boxes=1000, image_width=999, image_height=415, mode=xyxy)]
GeneralizedRCNN model forward


  2%|▏         | 2/100 [00:01<01:02,  1.58it/s]

...

 99%|█████████▉| 99/100 [00:29<00:00,  3.77it/s]

roi_head
self.score_thresh 0.5
reln 1
orig_features torch.Size([9, 4096])

img_id (98,)
proposal: [BoxList(num_boxes=1000, image_width=999, image_height=428, mode=xyxy)]
GeneralizedRCNN model forward


100%|██████████| 100/100 [00:29<00:00,  3.74it/s]

roi_head
self.score_thresh 0.5
reln 1
orig_features torch.Size([17, 4096])

img_id (99,)
proposal: [BoxList(num_boxes=1000, image_width=999, image_height=428, mode=xyxy)]


100%|██████████| 100/100 [00:29<00:00,  3.37it/s]


In [8]:
print(feature_list[0].shape)
print(feature_list[0][0].shape)
print(feature_list[1].shape)

print(origfeatlist[0].shape)
print(origfeatlist[0][0].shape)
print(origfeatlist[1].shape)



torch.Size([21, 4096])
torch.Size([4096])
torch.Size([11, 4096])
torch.Size([21, 4096])
torch.Size([4096])
torch.Size([11, 4096])


In [9]:
# print(len(origfeatlist))
# print(len(origfeatlist[0]))
# print(origfeatlist[0][0].shape)
# print(origfeatlist[0][1].shape)

100
21
torch.Size([4096])
torch.Size([4096])


In [10]:
print(len(idslist))
print(len(proplist))
print(len(feature_list))
print(len(outlist))

100
100
100
100


In [11]:
feature_list[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [34]:
# print(outlist[0][0].size)
# print(outlist[0][0].bbox.cpu().numpy())
# [[y * 2 for y in x ] for x in outlist[0][0].bbox.cpu().numpy()]

In [17]:
proplist[0]

AttributeError: 'list' object has no attribute 'get_field'

In [65]:
# img_info
# outlist[0][0].bbox.cpu().numpy()
for x1,y1, x2, y2 in outlist[0][0].bbox.cpu().numpy():
    print(x1)

89.10719
806.54193
562.2655
382.39548
0.0
511.0694
493.24127
238.73792
587.8737
129.71655
11.530899
825.43915
531.6215
185.00858
226.66487
463.1055
728.99866
470.52612
42.549225
140.65356
566.14355


In [70]:
# id_list= [x[0] for x in idslist]


bblist = []
orig_bblist = []
scaled_bblist = []
for info, output in zip(img_info, outlist):
    bbox = output[0].bbox.cpu().numpy()
    bblist.append(bbox)
    widthi, heighti = output[0].size
    width, height =  info["width"],  info["height"]
    scale = width/widthi
    orig_bbox = [[y * scale for y in x ] for x in bbox]
    orig_bblist.append(orig_bbox)
    scaled_bblist.append([np.array([x1/width, y1/height, x2/width, y2/height]) for x1,y1, x2, y2 in orig_bbox])


In [77]:
# np.amax(np.array(scaled_bblist))
# np.amax(np.array(scaled_bblist[0]))

0.9965327265742304

In [45]:
import numpy as np

feat_np = [x.cpu().numpy() for x in feature_list]
feat_np

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0.       , 0.       , 0.       , ..., 0.       , 0.5497424,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.0877497]], dtype=float32),
 array([[0.        , 0.07557935, 0.        , ..., 0.        , 0.10220677,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.029308  , 0.      

In [85]:
import pickle
import os
import numpy as np
os.remove('/home/suji/spring20/stuffs.pkl')

# d = {'feat_tensor': feature_list, 'feat_np': feat_np, 'proposal': proplist, 'img_id':id_list, 'orig_bb':orig_bblist, 'bb': bblist, 'scaled_bb': scaled_bblist}
d = { 'feat_np': feat_np, 'img_id':id_list, 'orig_bb':orig_bblist, 'bb': bblist, 'scaled_bb': scaled_bblist}
with open("/home/suji/spring20/stuffs.pkl", "wb") as f:
    pickle.dump(d, f)

In [84]:
d['proposal']

[[BoxList(num_boxes=1000, image_width=999, image_height=415, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=562, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=415, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=562, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=429, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=562, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=1000, image_height=421, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=428, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=1000, image_height=425, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=429, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=1000, image_height=425, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=999, image_height=562, mode=xyxy)],
 [BoxList(num_boxes=1000, image_width=1000, image_height=425, mode=xyxy)],
 [BoxList(num_boxes=1000, image_wi

```
#lmdb
import lmdb
import numpy as np
import tqdm


MAP_SIZE = 1099511627776

# infiles = glob.glob(os.path.join(args.features_dir, "*"))
# id_list = []
env = lmdb.open("rcnn_feat.lmdb", map_size=MAP_SIZE)
id_list= [x[0] for x in idslist]
with env.begin(write=True) as txn:
for output, feature, img_id in zip(outlist, feature_list, id_list):
    item = {}
    item[] = feature
    item[] = output[0]
    txn.put(img_id, pickle.dumps(item))
txn.put("keys".encode(), pickle.dumps(id_list))
# 
#     for infile in tqdm.tqdm(infiles):
#         reader = np.load(infile, allow_pickle=True)
#         item = {}
#         item["image_id"] = reader.item().get("image_id")
#         img_id = str(item["image_id"]).encode()
#         id_list.append(img_id)
#         item["image_h"] = reader.item().get("image_height")
#         item["image_w"] = reader.item().get("image_width")
#         item["num_boxes"] = reader.item().get("num_boxes")
#         item["boxes"] = reader.item().get("bbox")
#         item["features"] = 
#         txn.put(img_id, pickle.dumps(item))
#     txn.put("keys".encode(), pickle.dumps(id_list))
```


# visualize all proposals

In [15]:
# objectness
# data_loaders_val.get_img_info[image_ids[0]]
from PIL import Image
import os
import pandas as pd
df = pd.read_csv('/home/suji/spring20/filenames.csv')
filenames =[os.path.join("/home/suji/spring20/vilbert_beta/data/VCR/vcr1images",fn) for fn in list(df['img_fn'])]
img_info = []

for index in range(len(filenames)):
    img = Image.open(filenames[index]).convert("RGB")
    #if transforms is not None:
        #print("image being transformed")
        #img = transforms(img)
        #print("image transformed to ", type(img))
    #self.imgs.append(img)
    img_info.append({"width": img.size[0], "height":img.size[1]})


In [ ]:
proplist[0][0].size#.get_field('image_width')

In [ ]:
'''import json

f = open("detection_res.jsonl", "a")
for i, (fn, info, proposal, idnum) in enumerate(zip(filenames, img_info, proplist, idslist)):
    img_width = info["width"]
    img_height = info["height"]
    prop = proposal[0]
#     prop.resize((image_width, image_height))
    objectness = prop.get_field('objectness')
    predict_logits = prop.get_field('predict_logits')
    bbox = prop.bbox
    f.write(json.dumps({'img_id':idnum[0], 'objectness': objectness.cpu().numpy().tolist(), 'predict_logits':predict_logits.cpu().numpy().tolist(), 'bbox': bbox.cpu().numpy().tolist(), 'img_height':prop.size[1],'img_width':prop.size[0]}))
    f.write("\n")
f.close()'''

In [ ]:
# image_width = img_info["width"]
# image_height = img_info["height"]
# proposal[0].resize((image_width, image_height))

In [ ]:
# df

In [ ]:
# df2 = pd.read_json("detection_res.jsonl")

In [ ]:
with open("detection_res.jsonl", "r") as reader:
    lines = reader.readlines()

In [ ]:
# filenames

In [ ]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1[0] < bb1[2]
    assert bb1[1] < bb1[3]
    assert bb2[0] < bb2[2]
    assert bb2[1] < bb2[3]

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou
def valid_bb(bb1, bb2):
    return get_iou(bb1, bb2) < .3

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
def filter_bb(bbox, objectness):
    bb_list = []
    for bb, objness in zip(bbox, objectness):
        if objness >  0.85:
            if len(bb_list) == 0:
                bb_list.append(bb)
                continue
            if all([valid_bb(bb, other_bb) for other_bb in bb_list]):
                bb_list.append(bb)
    return bb_list
d = {}
for line in lines:
    entry = json.loads(line)
    img_id = entry['img_id']
    fn = filenames[img_id]
    bb_list = filter_bb(entry['bbox'], entry['objectness'])
    print(len(bb_list))
    d[img_id] = (bb_list, entry['img_width'], entry['img_height'])
    #     
    

In [ ]:
for i, (img_id, tup) in enumerate(d.items()):
#     print(img_info[i])
    fn = filenames[i]
    (bbox_list, widthi, heighti) = tup
    im = mpimg.imread(fn)
    height, width, _ = im.shape
    print(width/widthi )
    print(height/heighti)
    print(bbox_list)
    scale = width/widthi
    bbox_list = [[y * scale for y in x ]for x in bbox_list]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')
    for i, bbox in enumerate(bbox_list):
#         pred_classes_str = ind_to_classes[int(pred_classes[i])]

#         while pred_classes_str in object_name_list_pred:
#             obj_count_pred[int(pred_classes[i])] += 1
#             pred_classes_str = pred_classes_str + '_' + str(obj_count_pred[int(pred_classes[i])])
#         object_name_list_pred.append(pred_classes_str)
#         ax.text(bbox[0], bbox[1] - 2,
#                 pred_classes_str,
#                 bbox=dict(facecolor='green', alpha=0.5),
#                 fontsize=32, color='white')
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                        bbox[2] - bbox[0],
                        bbox[3] - bbox[1], fill=False,
                        edgecolor='green', linewidth=3.5)
        )

    ax.axis('off')
    fig.tight_layout()
    plt.show()
#     image_save_fn = sg_save_fn.replace("graph", "image") + ".{}".format(save_format) 
#     plt.savefig(image_save_fn)
    plt.close()

In [ ]:
import torch
import json
import h5py
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw
ndetected_origin_path = '/home/suji/checkpoints/causal-motifs-sgcls-exmp/inference/vcr_test/'
detected_origin_result = torch.load(ndetected_origin_path + 'eval_results.pytorch')
detected_info = json.load(open(ndetected_origin_path + 'visual_info.json'))

In [ ]:
detected_origin_result.keys()

In [ ]:
detected_info